# Mount Google Drive for Colab Use Only

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Import the Necessary Libraries

In [2]:
import scipy
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
from glob import glob
from PIL import Image

Using TensorFlow backend.


# Data Related Functions

In [0]:
def load_train(sizeOfBatch=1):
  traindir = '/content/drive/My Drive/Colab Notebooks/CycleGANs/ML_Dataset_5/train/' #Enter your own directory of TRAINING images here
  res = (256,256)
  numBatches = int(len(glob(traindir+'*'))/sizeOfBatch)
  for batch in range(numBatches-1):
    currBatch = glob(traindir+'*')[sizeOfBatch*batch:sizeOfBatch*(batch+1)]
    imgA = []
    imgB = []
    for i in currBatch:         #Enter the name of your first and last images here [216.jpg and 349.jpg] in this case
      img = scipy.misc.imread(i, mode='RGB').astype(np.float)#plt.imread(traindir+str(i)+'.jpg')                #Edit the format of images if neccessary
      width = img.shape[1]
      a = scipy.misc.imresize(img[:,:width//2,:],res)
      b = scipy.misc.imresize(img[:,width//2:,:],res)
      if np.random.random() > 0.5:
        a = np.fliplr(a)
        b = np.fliplr(b)
      imgA.append(a)
      imgB.append(b)
    imgsA = normalizeIMG(imgA)
    imgsB = normalizeIMG(imgB)
    yield imgsA, imgsB

def load_test():
  testdir = '/content/drive/My Drive/Colab Notebooks/CycleGANs/ML_Dataset_5/test/'  #Enter your own directory of TEST images here
  testA = []
  testB = []
  res = (256,256)
  testIMGs = np.random.choice(glob(testdir+'*'), 1)
  for i in testIMGs:
    img = scipy.misc.imread(i, mode='RGB').astype(np.float)
    width = img.shape[1]
    testA.append(scipy.misc.imresize(img[:,:width//2,:],res))
    testB.append(scipy.misc.imresize(img[:,width//2:,:],res))
  testsA = normalizeIMG(testA)
  testsB = normalizeIMG(testB)
  return testsA, testsB

def normalizeIMG(img):
  return np.array(img)/127.5 - 1

# Model Related Functions

In [0]:
def build_generator():
  
    InIMG = Input(shape=img_shape)
    
    down1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(InIMG)
    down1 = LeakyReLU(alpha=0.2)(down1)
    
    down2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(down1)
    down2 = LeakyReLU(alpha=0.2)(down2)
    down2 = BatchNormalization(momentum=0.8)(down2)
    
    down3 = Conv2D(256, kernel_size=4, strides=2, padding='same')(down2)
    down3 = LeakyReLU(alpha=0.2)(down3)
    down3 = BatchNormalization(momentum=0.8)(down3)
    
    down4 = Conv2D(512, kernel_size=4, strides=2, padding='same')(down3)
    down4 = LeakyReLU(alpha=0.2)(down4)
    down4 = BatchNormalization(momentum=0.8)(down4)
    
    down5 = Conv2D(512, kernel_size=4, strides=2, padding='same')(down4)
    down5 = LeakyReLU(alpha=0.2)(down5)
    down5 = BatchNormalization(momentum=0.8)(down5)
    
    down6 = Conv2D(512, kernel_size=4, strides=2, padding='same')(down5)
    down6 = LeakyReLU(alpha=0.2)(down6)
    down6 = BatchNormalization(momentum=0.8)(down6)
    
    down7 = Conv2D(512, kernel_size=4, strides=2, padding='same')(down6)
    down7 = LeakyReLU(alpha=0.2)(down7)
    down7 = BatchNormalization(momentum=0.8)(down7)

    
    
    up1 = UpSampling2D(size=2)(down7)
    up1 = Conv2D(512, kernel_size=4, strides=1, padding='same', activation='relu')(up1)
    up1 = BatchNormalization(momentum=0.8)(up1)
    up1 = Concatenate()([up1, down6])
    
    up2 = UpSampling2D(size=2)(up1)
    up2 = Conv2D(512, kernel_size=4, strides=1, padding='same', activation='relu')(up2)
    up2 = BatchNormalization(momentum=0.8)(up2)
    up2 = Concatenate()([up2, down5])
    
    up3 = UpSampling2D(size=2)(up2)
    up3 = Conv2D(512, kernel_size=4, strides=1, padding='same', activation='relu')(up3)
    up3 = BatchNormalization(momentum=0.8)(up3)
    up3 = Concatenate()([up3, down4])
    
    up4 = UpSampling2D(size=2)(up3)
    up4 = Conv2D(256, kernel_size=4, strides=1, padding='same', activation='relu')(up4)
    up4 = BatchNormalization(momentum=0.8)(up4)
    up4 = Concatenate()([up4, down3])
    
    up5 = UpSampling2D(size=2)(up4)
    up5 = Conv2D(128, kernel_size=4, strides=1, padding='same', activation='relu')(up5)
    up5 = BatchNormalization(momentum=0.8)(up5)
    up5 = Concatenate()([up5, down2])
    
    up6 = UpSampling2D(size=2)(up5)
    up6 = Conv2D(64, kernel_size=4, strides=1, padding='same', activation='relu')(up6)
    up6 = BatchNormalization(momentum=0.8)(up6)
    up6 = Concatenate()([up6, down1])
    
    up7 = UpSampling2D(size=2)(up6)
    OutIMG = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(up7)
    
    return Model(InIMG, OutIMG)
    
def build_discriminator():

    A = Input(shape=img_shape)
    B = Input(shape=img_shape)

    AB = Concatenate(axis=-1)([A, B])

    down1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(AB)
    down1 = LeakyReLU(alpha=0.2)(down1)
    
    down2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(down1)
    down2 = LeakyReLU(alpha=0.2)(down2)
    down2 = BatchNormalization(momentum=0.8)(down2)
    
    down3 = Conv2D(256, kernel_size=4, strides=2, padding='same')(down2)
    down3 = LeakyReLU(alpha=0.2)(down3)
    down3 = BatchNormalization(momentum=0.8)(down3)
    
    down4 = Conv2D(512, kernel_size=4, strides=2, padding='same')(down3)
    down4 = LeakyReLU(alpha=0.2)(down4)
    down4 = BatchNormalization(momentum=0.8)(down4)
    
    discOut = Conv2D(1, kernel_size=4, strides=1, padding='same')(down4)
    
    return Model([A, B], discOut)
  
def train(epochs, sizeOfBatch=1, testAfter=50):

    start_time = datetime.datetime.now()

    valid = np.ones((sizeOfBatch,) + disc_outShape)
    fake = np.zeros((sizeOfBatch,) + disc_outShape)
    
    for currEpoch in range(epochs):
        for i, (A, B) in enumerate(load_train(sizeOfBatch)):

            fakeIMG = generator.predict(B)

            disc_loss_real = discriminator.train_on_batch([A, B], valid)
            disc_loss_fake = discriminator.train_on_batch([fakeIMG, B], fake)
            disc_loss = 0.5 * np.add(disc_loss_real, disc_loss_fake)

            gen_loss = combined.train_on_batch([A, B], [valid, A])

            elapsed_time = datetime.datetime.now() - start_time
            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (currEpoch, epochs,
                                                                    i, numBatches,
                                                                    disc_loss[0], 100*disc_loss[1],
                                                                    gen_loss[0],
                                                                    elapsed_time))

            if i % testAfter == 0:
                print("Testing Image at Iteration No."+str(i))
                test(currEpoch, i)

def test(epoch, batch):
    imgdir = '/content/drive/My Drive/ML Project/'
    os.makedirs(imgdir + 'imagesFin/%s' % dataset_name, exist_ok=True)
    
    imgs_A, imgs_B = load_test()
    fake_A = generator.predict(imgs_B)

    fake = np.concatenate([fake_A, imgs_B, imgs_A])
    A = np.concatenate([imgs_A, fake_A, imgs_B])
    B = np.concatenate([imgs_B, fake_A, imgs_A])
    print(fake_A.shape)
    # Rescale images 0 - 1
    fake = 0.5 * fake + 0.5
    A = 0.5 * A + 0.5
    B = 0.5 * B + 0.5
    
    fake1 = (fake[0]*255)
    A1 = A[0]*255
    B1 = B[0]*255
    Image.fromarray(fake1.astype('uint8')).save(imgdir + "imagesFin/%s/%d_%d_fake.png" % (dataset_name, epoch, batch))
    Image.fromarray(A1.astype('uint8')).save(imgdir + "imagesFin/%s/%d_%d_A.png" % (dataset_name, epoch, batch))
    Image.fromarray(B1.astype('uint8')).save(imgdir + "imagesFin/%s/%d_%d_B.png" % (dataset_name, epoch, batch))


# Initialize the Model

In [0]:
from PIL import Image
img_rows = 256
img_cols = 256
channels = 3
img_shape = (img_rows, img_cols, channels)
img_res=(img_rows, img_cols)
dataset_name = 'ML_Dataset_5'

imgdir = '/content/drive/My Drive/Colab Notebooks/CycleGANs'

path = glob(imgdir + '/%s/train/*' % (dataset_name))
numBatches = int(len(path))
patch = int(img_rows / 2**4)
disc_outShape = (patch, patch, 1)

optimizer = Adam(0.0002, 0.5)

discriminator = build_discriminator()
discriminator.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])

generator = build_generator()

img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)

fake_A = generator(img_B)
discriminator.trainable = False
valid = discriminator([fake_A, img_B])

combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
combined.compile(loss=['mse', 'mae'],
                      loss_weights=[1, 100],
                      optimizer=optimizer)

# Training and Testing the Model

In [52]:
train(epochs=10, sizeOfBatch=1, testAfter=50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between 

[Epoch 0/10] [Batch 0/134] [D loss: 3.634269, acc:  16%] [G loss: 73.228333] time: 0:01:06.653315
Testing Image at Iteration No.0
(1, 256, 256, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[Epoch 0/10] [Batch 1/134] [D loss: 9.263795, acc:  17%] [G loss: 57.153648] time: 0:01:07.219920
[Epoch 0/10] [Batch 2/134] [D loss: 8.986752, acc:  21%] [G loss: 50.083134] time: 0:01:07.583372
[Epoch 0/10] [Batch 3/134] [D loss: 7.457245, acc:  19%] [G loss: 51.428093] time: 0:01:07.939492
[Epoch 0/10] [Batch 4/134] [D loss: 3.634562, acc:  18%] [G loss: 37.666862] time: 0:01:08.292941
[Epoch 0/10] [Batch 5/134] [D loss: 2.628344, acc:  26%] [G loss: 36.574558] time: 0:01:08.645688
[Epoch 0/10] [Batch 6/134] [D loss: 2.583096, acc:  22%] [G loss: 34.790089] time: 0:01:09.003503
[Epoch 0/10] [Batch 7/134] [D loss: 3.333271, acc:  25%] [G loss: 29.965353] time: 0:01:09.355869
[Epoch 0/10] [Batch 8/134] [D loss: 2.045856, acc:  51%] [G loss: 25.767807] time: 0:01:09.710903
[Epoch 0/10] [Batch 9/134] [D loss: 1.462942, acc:  30%] [G loss: 22.385681] time: 0:01:10.069833
[Epoch 0/10] [Batch 10/134] [D loss: 0.895923, acc:  48%] [G loss: 20.873041] time: 0:01:10.429182
[Epoch 0/10] [Batch

KeyboardInterrupt: ignored

# Saving Model Weights for Future Use

In [0]:
weightsdir = '/content/drive/My Drive/ML Project/images50/'
combined.save_weights(weightsdir+'combined_weights.h5')
discriminator.save_weights(weightsdir+'discriminator_weights.h5')
generator.save_weights(weightsdir+'generator_weights.h5')